In [ ]:
import preamble
import os
import numpy as np
from numpy.typing import NDArray
from phrase_trial_data import PhraseTrialData

DIRECTORY = "phrase_trial_data"
FORCE_THRESHOLD = 7

In [ ]:
data = PhraseTrialData("move up", user_id=1)
data.append(time=0, dt=0.0, position=np.array([0, 0, 0]), velocity=np.array([0, 0, 0]), force=np.array([0, 0, 0]))
data.save()

In [ ]:
def compute_average_force(data: PhraseTrialData) -> NDArray:
    force = np.array(data.force)
    dt = np.array(data.dt)

    filtered_force = force * (force > FORCE_THRESHOLD)
    filtered_dt = dt * (force > FORCE_THRESHOLD)

    average_force = np.sum(filtered_force * filtered_dt, axis=0) / np.sum(filtered_dt)
    return average_force

for file_name in os.listdir(DIRECTORY):
    if not file_name.endswith('.pkl'):
        continue

    file_path = os.path.join(DIRECTORY, file_name)
    data = PhraseTrialData.load(file_path)

    direction = data.get_direction()
    baseline_file_path = os.path.join(DIRECTORY, f'{data.user_id}__{direction}.pkl')
    baseline_data = PhraseTrialData.load(baseline_file_path)

    average_force = compute_average_force(data)
    baseline_force = compute_average_force(data)

    emphasis = np.linalg.norm(average_force) / np.linalg.norm(baseline_force)
    average_direction = average_force / np.linalg.norm(average_force)
    
    row = [data.phrase, data.user_id, average_force, emphasis, average_direction]